In [3]:
import sys
print(sys.executable)

import torch
print(torch.__version__)

H:\Conda\envs\pytorch-env\python.exe
2.2.2+cu121


In [4]:
import torch

if torch.cuda.is_available():
    print("CUDA is available. Here are the GPU details:")
    print(torch.cuda.get_device_name(0))
else:
    print("CUDA is not available.")

CUDA is available. Here are the GPU details:
NVIDIA GeForce RTX 3080


In [5]:
import numpy as np
import pandas as pd

In [6]:
df_user_score=pd.read_csv('./users-score-2023.csv')
df_user_score.head()

,user_id,Username,anime_id,Anime Title,rating
0,1,Xinil,21,One Piece,9
1,1,Xinil,48,.hack//Sign,7
2,1,Xinil,320,A Kite,5
3,1,Xinil,49,Aa! Megami-sama!,8
4,1,Xinil,304,Aa! Megami-sama! Movie,8


In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.utils import shuffle

In [8]:
# shuffle the data since was ordered by the user id
df_user_score = shuffle(df_user_score, random_state=66)
# do the encoding
# Create a MinMaxScaler object
scaler = MinMaxScaler(feature_range=(0, 1))
# Scale the 'score' column between 0 and 1
df_user_score['scaled_score'] = scaler.fit_transform(df_user_score[['rating']])
user_encoder = LabelEncoder()
anime_encoder = LabelEncoder()
df_user_score['anime_id_encoded'] = anime_encoder.fit_transform(df_user_score['anime_id'])
df_user_score['user_id_encoded'] = user_encoder.fit_transform(df_user_score['user_id'])
df_user_score.head()

,user_id,Username,anime_id,Anime Title,rating,scaled_score,anime_id_encoded,user_id_encoded
686198,8160,dianakitsune,65,Rozen Maiden: Träumend,9,0.888889,46,5251
17015808,496175,AB_PB,3655,Nabari no Ou,9,0.888889,3186,187528
6376853,102759,the_shade,27631,God Eater,7,0.666667,8879,63569
2707547,36746,Arata84,22297,Fate/stay night: Unlimited Blade Works,8,0.777778,8215,24296
3203122,44347,roybarboza96,4720,White Album,9,0.888889,3775,29106


In [9]:
# X is the features used for prediction
# y is the target
X = df_user_score[['user_id_encoded','anime_id_encoded']].values
y = df_user_score['scaled_score'].values
print("Shape of X:", X.shape)
print("Shape of y:", y.shape)

Shape of X: (24325191, 2)
Shape of y: (24325191,)


In [13]:
# split training data and validation data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

X_train = X_train.astype('int32')
X_test = X_test.astype('int32')
print("Number of samples in the training set:", len(y_train))
print("Number of samples in the test set:", len(y_test))

Number of samples in the training set: 19460152
Number of samples in the test set: 4865039


In [92]:
# Define the model architecture
class CollaborativeFilteringModel(nn.Module):
    def __init__(self, num_users, num_animes, embedding_size):
        super(CollaborativeFilteringModel, self).__init__()
        self.user_embedding = nn.Embedding(num_users, embedding_size)
        self.anime_embedding = nn.Embedding(num_animes, embedding_size)
        self.dense = nn.Linear(1, 1)  # Modify the input size to 1
        self.sigmoid = nn.Sigmoid()

    def forward(self, user_input, anime_input):
        user_embedded = self.user_embedding(user_input)
        anime_embedded = self.anime_embedding(anime_input)
        dot_product = torch.mul(user_embedded, anime_embedded).sum(dim=-1)
        dot_product = dot_product.unsqueeze(1)  # Add an extra dimension
        dense_output = self.dense(dot_product)
        output = self.sigmoid(dense_output)
        return output

In [95]:
num_users = df_user_score['user_id_encoded'].max() + 1
num_animes = df_user_score['anime_id_encoded'].max() + 1
embedding_size = 128

# Check if GPU is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

# Create an instance of the model
model = CollaborativeFilteringModel(num_users, num_animes, embedding_size).to(device)

cuda


In [96]:
# Define the loss function and optimizer
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters())

In [98]:
# Training loop
num_epochs = 20
batch_size = 10000

for epoch in range(num_epochs):
    for i in range(0, len(X_train), batch_size):
        batch_users = torch.LongTensor(X_train[i:i+batch_size, 0]).to(device)
        batch_animes = torch.LongTensor(X_train[i:i+batch_size, 1]).to(device)
        batch_ratings = torch.FloatTensor(y_train[i:i+batch_size]).to(device)

        # Forward pass
        outputs = model(batch_users, batch_animes)
        loss = criterion(outputs.squeeze(), batch_ratings)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # Print the loss for every epoch
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")

# Evaluate the model on the test set
with torch.no_grad():
    test_users = torch.LongTensor(X_test[:, 0]).to(device)
    test_animes = torch.LongTensor(X_test[:, 1]).to(device)
    test_ratings = torch.FloatTensor(y_test).to(device)

    test_outputs = model(test_users, test_animes)
    test_loss = criterion(test_outputs.squeeze(), test_ratings)

print(f"Test Loss: {test_loss.item():.4f}")

Epoch [1/20], Loss: 0.5984
Epoch [2/20], Loss: 0.5822
Epoch [3/20], Loss: 0.5825
Epoch [4/20], Loss: 0.5812
Epoch [5/20], Loss: 0.5792
Epoch [6/20], Loss: 0.5754
Epoch [7/20], Loss: 0.5706
Epoch [8/20], Loss: 0.5622
Epoch [9/20], Loss: 0.5575
Epoch [10/20], Loss: 0.5668
Epoch [11/20], Loss: 0.5955
Epoch [12/20], Loss: 0.6279
Epoch [13/20], Loss: 0.6433
Epoch [14/20], Loss: 0.6433
Epoch [15/20], Loss: 0.6338
Epoch [16/20], Loss: 0.6197
Epoch [17/20], Loss: 0.6034
Epoch [18/20], Loss: 0.5858
Epoch [19/20], Loss: 0.5673
Epoch [20/20], Loss: 0.5482
Test Loss: 0.5896


In [99]:
torch.save(model.state_dict(), 'model_weights.pth')